In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_ters = 10000
learning_rate=3e-4
evar_iters =250

cpu


In [77]:
with open ("/Users/sebascordova/Desktop/Code proyects/wizard_of_oz.txt", 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [3]:

torch_1 = torch.rand(100, 100, 100, 100, device=device)
torch_2 = torch.rand(100, 100, 100, 100, device=device)


start_time = time.time()

result = torch.matmul(torch_1, torch_2)
end_time = time.time()
duration = end_time - start_time

print(f"Duración: {duration} segundos")

cpu
Duración: 0.48589515686035156 segundos


In [83]:

# Mapeo de los carácteres a un indice y su posterior mapeo para cada indice entero de vuelta a su carácter correspondiente.
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

#Codificación y decodificación
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    contexto = x[:t+1]
    tarjet = y[t]
    print('When input is', contexto, ' tarjet is ', tarjet)

When input is tensor([91])  tarjet is  tensor(48)
When input is tensor([91, 48])  tarjet is  tensor(65)
When input is tensor([91, 48, 65])  tarjet is  tensor(62)
When input is tensor([91, 48, 65, 62])  tarjet is  tensor(1)
When input is tensor([91, 48, 65, 62,  1])  tarjet is  tensor(44)
When input is tensor([91, 48, 65, 62,  1, 44])  tarjet is  tensor(75)
When input is tensor([91, 48, 65, 62,  1, 44, 75])  tarjet is  tensor(72)
When input is tensor([91, 48, 65, 62,  1, 44, 75, 72])  tarjet is  tensor(67)


In [88]:
batch_size

4

In [91]:
len(data) - batch_size

252603

In [90]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):

    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+batch_size +1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([175681,  11722, 141855,  67005])
inputs:
tensor([[75, 62,  1, 71, 72, 77,  1, 75],
        [ 0, 65, 66, 70, 76, 62, 69, 63],
        [58, 77, 62, 61, 12,  1, 58, 71],
        [76,  1, 82, 72, 78,  1, 73, 69]])
targets:
tensor([[62,  1, 71, 72],
        [65, 66, 70, 76],
        [77, 62, 61, 12],
        [ 1, 82, 72, 78]])


In [80]:
class BiogramLanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BiogramLanguagemodel(vocab_size)
m = model.to(device)


context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



/’INe,x&VT7 ™rYHho-*UUb/0mi5™l:HmJ9yNV•q”hX7ka3U/V Q‘Fe_y8"gyaboi2W6D3™LU™X4hN]“UXsXqKEz:Q*n?Gmv_"COa;“:!QHOTM*A-ESuMnZaeb“PbOFX?cO™•,“z00o0#﻿Ar—s'6•t[i﻿’N[Q‘fRABM-zfp’6;Vmy9*AY58K$iy]73KMl'm$z/Vq#TV$R"﻿gnr.HuVd4‘QC'Fpv”)z[‘mZZ?)mBRp&%?ZP*iuP”??ig!PMs“j﻿'T2-KTeQ*ou[/ZBpnxFbWYeHP9_C]jNm?lT9ccZVXH#B3nLmx$O •&Iq*Jrj_‘vO/SYN6L.xJ;p™/-Hff0•!-"9k(:JI]:$'Hup]‘T'?NT7vM4zk“sO?fXNb$“(] i!KMlnu5]//
k#'0L1AYW(g[zl143csP5M/CF*J-sC*2k6•Wu])zIkQ*﻿?6o2xo2VXZ•t1S•q”Zsq_Loh“Vw:jKN
O);ZdOsSkry2ya;™Es:;dj‘AYxU#hoih


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")


    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())